In [1]:
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies')

import alchemy_tools as at
from geometry_euston import abc_to_hmatrix, distance_pbc
from parse_cube_files import CUBE

import numpy as np
import itertools as it

Calculation of the distance between a nucleus and a group of vectors represented on a grid using the minimal image convention

In [2]:
def distance_MIC(pos_nuc, meshgrid, h_matrix):
    """
    calculates the distance between the position of the nucleus and the nearest image of a gridpoint
    
    pos_nuc: position of nucleus
    meshgrid: meshgrid containing x,y,z components of every gridpoint
    h_matrix: needed for calculation of MIC distance
    """
    
    distance = np.zeros((meshgrid[0].shape))
    
    for idx0 in range(meshgrid[0].shape[0]):
        for idx1 in range(meshgrid[0].shape[1]):
            for idx2 in range(meshgrid[0].shape[2]):
                distance[idx0][idx1][idx2] = distance_pbc(pos_nuc, np.array([meshgrid[0][idx0][idx1][idx2], meshgrid[1][idx0][idx1][idx2], meshgrid[2][idx0][idx1][idx2]]), h_matrix)
    
    return(distance)

In [3]:
def distance_MIC2(pos_nuc, meshgrid, h_matrix):
    """
    calculates the distance between the position of the nucleus and the nearest image of a gridpoint
    works so far only for cubic symmetry
    
    pos_nuc: position of nucleus
    meshgrid: meshgrid containing x,y,z components of every gridpoint
    h_matrix: needed for calculation of MIC distance
    :return: distance between pos_nuc and every gridpoint
    :rtype: numpy array of shape meshgrid.shape
    """
    
    hinv = np.linalg.inv(h_matrix)
    a_t = np.dot(hinv, pos_nuc)
    
    # calculate product of h_matrix and grid componentwise
    b_t_x = hinv[0][0]*meshgrid[0]
    b_t_y = hinv[1][1]*meshgrid[1]
    b_t_z = +hinv[2][2]*meshgrid[2]
    
    t_12_x = b_t_x - a_t[0]
    t_12_y = b_t_y - a_t[1]
    t_12_z = b_t_z - a_t[2]
    
    t_12_x -= np.round(t_12_x)
    t_12_y -= np.round(t_12_y)
    t_12_z -= np.round(t_12_z)
    
    x = np.power(h_matrix[0][0]*t_12_x, 2)
    y = np.power(h_matrix[1][1]*t_12_y, 2)
    z = np.power(h_matrix[2][2]*t_12_z, 2)
    
    return(np.sqrt(x+y+z))

## Test function to calculate distance to closest image
Create a 2D grid and calculate the distances for one point manually (selected point shown in plot)
2D version of MIC returns same values as manual calculation 

In [4]:
def distance_MIC2_2D(pos_nuc, meshgrid, h_matrix):
    """
    calculates the distance between the position of the nucleus and the nearest image of a gridpoint
    works so far only for cubic symmetry
    
    pos_nuc: position of nucleus
    meshgrid: meshgrid containing x,y,z components of every gridpoint
    h_matrix: needed for calculation of MIC distance
    :return: distance between pos_nuc and every gridpoint
    :rtype: numpy array of shape meshgrid.shape
    """
    
    hinv = np.linalg.inv(h_matrix)
    a_t = np.dot(hinv, pos_nuc)
    
    # calculate product of h_matrix and grid componentwise
    b_t_x = hinv[0][0]*meshgrid[0]
    b_t_y = hinv[1][1]*meshgrid[1]
    
    t_12_x = b_t_x - a_t[0]
    t_12_y = b_t_y - a_t[1]
    
    t_12_x -= np.round(t_12_x)
    t_12_y -= np.round(t_12_y)
    
    x = np.power(h_matrix[0][0]*t_12_x, 2)
    y = np.power(h_matrix[1][1]*t_12_y, 2)
    
    return(np.sqrt(x+y))

In [14]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt

L = 3
p0 = (0, L), (L, 2*L)
p1 = (L, 2*L), (L, 2*L)
p2 = (L, 2*L), (0, L)
p3 = (L, 2*L), (-L, 0)
p4 = (0, L), (-L, 0)
p5 = (-L, 0), (-L, 0)
p6 = (-L, 0), (0, L)
p7 = (-L, 0), (L, 2*L)

p = [p0, p1, p2, p3, p4, p5, p6, p7]

fig, ax = plt.subplots(1,1)

# create boxes around cell of interest
for el in p:
    xx, yy = np.meshgrid(np.linspace(el[0][0], el[0][1]-1, L), np.linspace(el[1][0], el[1][1]-1, L))
    ax.scatter(xx, yy, color='#1f77b4')

# add cell of interest
x_test = np.linspace(0, L-1, L)
xx, yy = np.meshgrid(x_test, x_test)
ax.scatter(xx, yy, color='#ff7f0e')

# add boxes around every cell
verticals = [-3, 0, 3, 6]

for l in verticals:
    ax.plot(np.full(2, l), np.array([-3, 6]), '--', color = 'grey')
    ax.plot(np.array([-3, 6]), np.full(2, l), '--', color = 'grey')

# point for which distance to grid points shall be calculated
nuc_pos = np.array([0.2, 0.4])
ax.scatter([0.2], [0.4], marker='x', color='black')

# box around point nuc_pos used for calcualtion of distance with mic
ax.plot(np.array([-1, 2]), np.array([-1, -1]), '--', color = 'red')
ax.plot(np.array([-1, 2]), np.array([2, 2]), '--', color = 'red')
ax.plot(np.array([-1, -1]), np.array([-1, 2]), '--', color = 'red')
ax.plot(np.array([2, 2]), np.array([-1, 2]), '--', color = 'red')

In [15]:
distances = np.zeros(len(list(it.product((-1, 0, 1), repeat=2))))
for idx,i in enumerate(it.product((-1, 0, 1), repeat=2)):
    grid_vector = np.array([i[0], i[1]])
    distances[idx] = np.linalg.norm(grid_vector-nuc_pos)
d_grid = distances.reshape((3,3)).T
d_grid = np.roll(d_grid, -1, axis=0)
d_grid = np.roll(d_grid, -1, axis=1)
d_grid

array([[0.4472136 , 0.89442719, 1.26491106],
       [0.63245553, 1.        , 1.34164079],
       [1.41421356, 1.61245155, 1.84390889]])

In [7]:
# compare to MIC function
h_matrix = np.array([[3.0, 0.0], [0.0, 3.0]])
d_MIC2_2D = distance_MIC2_2D(nuc_pos, (xx, yy), h_matrix)

In [8]:
np.allclose(d_grid, d_MIC2_2D)

True

In [9]:
d_grid-d_MIC2_2D

array([[ 0.00000000e+00,  0.00000000e+00, -2.22044605e-16],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -2.22044605e-16]])

## Construction of H-matrix
compare output of abc_to_hmatrix to manually constructed h-matrix

In [10]:
cube = CUBE('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212/cube-files/ve_04.cube')

In [11]:
h_matrix = np.array([cube.X*cube.NX, cube.Y*cube.NY, cube.Z*cube.NZ])
h_matrix


array([[37.794575,  0.      ,  0.      ],
       [ 0.      , 37.794575,  0.      ],
       [ 0.      ,  0.      , 37.794575]])

In [12]:
abc_to_hmatrix((cube.X*cube.NX)[0], (cube.X*cube.NX)[0], (cube.X*cube.NX)[0], 90, 90, 90)

array([[3.77945750e+01, 2.31425026e-15, 2.31425026e-15],
       [0.00000000e+00, 3.77945750e+01, 2.31425026e-15],
       [0.00000000e+00, 0.00000000e+00, 3.77945750e+01]])

## Faster version to calculate MIC distance
compare fast and slow version

In [13]:
h_matrix = np.array([cube.X*cube.NX, cube.Y*cube.NY, cube.Z*cube.NZ])
h_matrix
meshgrid = cube.get_grid()
# hinv = np.linalg.inv(h_matrix)
# a = cube.atoms[0][1:4]
# a_t = np.dot(hinv, a)

### time versions

In [21]:
%timeit distances_mic = distance_MIC(cube.atoms[0][1:4], meshgrid, h_matrix)

1min 44s ± 1.75 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit distance2 = distance_MIC2(cube.atoms[0][1:4], meshgrid, h_matrix)

250 ms ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### compare output of versions

In [48]:
distances_mic = distance_MIC(cube.atoms[0][1:4], meshgrid, h_matrix)
distance_mic2 = distance_MIC2(cube.atoms[0][1:4], meshgrid, h_matrix)

In [49]:
np.allclose(distances_mic,distance_mic2)

True